# MPIA Arxiv on Deck 2

Contains the steps to produce the paper extractions.

In [1]:
# Imports
import os
from IPython.display import Markdown, display
from tqdm.notebook import tqdm
import warnings
from PIL import Image 

# requires arxiv_on_deck_2

from arxiv_on_deck_2.arxiv2 import (get_new_papers, 
                                    get_paper_from_identifier,
                                    retrieve_document_source, 
                                    get_markdown_badge)
from arxiv_on_deck_2 import (latex,
                             latex_bib,
                             mpia,
                             highlight_authors_in_list)

# Sometimes images are really big
Image.MAX_IMAGE_PIXELS = 1000000000 

In [2]:
# Some useful definitions.

class AffiliationWarning(UserWarning):
    pass

class AffiliationError(RuntimeError):
    pass

def validation(source: str):
    """Raises error paper during parsing of source file
    
    Allows checks before parsing TeX code.
    
    Raises AffiliationWarning
    """
    check = mpia.affiliation_verifications(source, verbose=True)
    if check is not True:
        raise AffiliationError("mpia.affiliation_verifications: " + check)

        
warnings.simplefilter('always', AffiliationWarning)


def get_markdown_qrcode(paper_id: str):
    """ Generate a qrcode to the arxiv page using qrserver.com
    
    :param paper: Arxiv paper
    :returns: markdown text
    """
    url = r"https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="
    txt = f"""<img src={url}"https://arxiv.org/abs/{paper_id}">"""
    txt = '<div id="qrcode">' + txt + '</div>'
    return txt

## get list of arxiv paper candidates

We use the MPIA mitarbeiter list webpage from mpia.de to get author names
We then get all new papers from Arxiv and match authors

In [3]:
# deal with the author list and edge cases of people that cannot be consistent on their name  

def filter_non_scientists(name: str) -> bool:
    """ Loose filter on expected authorships

    removing IT, administration, technical staff
    :param name: name
    :returns: False if name is not a scientist
    """
    remove_list = ['Wolf', 'Licht', 'Binroth', 'Witzel', 'Jordan',
                   'Zähringer', 'Scheerer', 'Hoffmann', 'Düe',
                   'Hellmich', 'Enkler-Scharpegge', 'Witte-Nguy',
                   'Dehen', 'Beckmann', 'Jager', 'Jäger'
                  ]

    for k in remove_list:
        if k in name:
            return False
    return True

def add_author_to_list(author_list: list) -> list:
    """ Add author to list if not already in list
    
    :param author: author name
    :param author_list: list of authors
    :returns: updated list of authors
    """
    add_list = ['T. Henning']

    for author in add_list:
        if author not in author_list:
            author_list.append(author)
    return author_list

# get list from MPIA website
# filter for non-scientists (mpia.get_mpia_mitarbeiter_list() does some filtering)
mpia_authors = [k[1] for k in mpia.get_mpia_mitarbeiter_list() if filter_non_scientists(k[1])]
# add some missing author because of inconsistencies in their MPIA name and author name on papers
mpia_authors = add_author_to_list(mpia_authors)

In [4]:
new_papers = get_new_papers()
# add manual references
add_paper_refs = []
new_papers.extend([get_paper_from_identifier(k) for k in add_paper_refs])

candidates = []
for paperk in new_papers:
    # Check author list with their initials
    normed_author_list = [mpia.get_initials(k) for k in paperk['authors']]
    hl_authors = highlight_authors_in_list(normed_author_list, mpia_authors, verbose=True)
    matches = [(hl, orig) for hl, orig in zip(hl_authors, paperk['authors']) if 'mark' in hl]
    paperk['authors'] = hl_authors
    if matches:
        # only select paper if an author matched our list
        candidates.append(paperk)
print("""Arxiv has {0:,d} new papers today""".format(len(new_papers)))        
print("""          {0:,d} with possible author matches""".format(len(candidates)))

D. Spaeth  ->  D. Spaeth  |  ['D. Spaeth']
H. Jiang  ->  H. Jiang  |  ['H. Jiang']
Arxiv has 60 new papers today
          2 with possible author matches


# Parse sources and generate relevant outputs

From the candidates, we do the following steps:
* get their tarball from ArXiv (and extract data)
* find the main .tex file: find one with \documentclass{...} (sometimes it's non trivial)
* Check affiliations with :func:`validation`, which uses :func:`mpia.affiliation_verifications`
* If passing the affiliations: we parse the .tex source
   * inject sub-documents into the main (flatten the main document)
   * parse structure, extract information (title, abstract, authors, figures...)
   * handles `\graphicspath` if provided
* Generate the .md document.

In [5]:
documents = []
failed = []
for paper in tqdm(candidates):
    # debug crap
    paper['identifier'] = paper['identifier'].lower().replace('arxiv:', '').replace(r'\n', '').strip()
    paper_id = paper['identifier']
    
    folder = f'tmp_{paper_id}'

    try:
        if not os.path.isdir(folder):
            folder = retrieve_document_source(f"{paper_id}", f'tmp_{paper_id}')
        
        try:
            doc = latex.LatexDocument(folder, validation=validation)    
        except AffiliationError as affilerror:
            msg = f"ArXiv:{paper_id:s} is not an MPIA paper... " + str(affilerror)
            failed.append((paper, "affiliation error: " + str(affilerror) ))
            continue
        
        # Hack because sometimes author parsing does not work well
        if (len(doc.authors) != len(paper['authors'])):
            doc._authors = paper['authors']
        else:
            # highlight authors (FIXME: doc.highlight_authors)
            # done on arxiv paper already
            doc._authors = highlight_authors_in_list(
                [mpia.get_initials(k) for k in doc.authors], 
                mpia_authors, verbose=True)
        if (doc.abstract) in (None, ''):
            doc._abstract = paper['abstract']
            
        doc.comment = (get_markdown_badge(paper_id) + 
                       "<mark>Appeared on: " + paper['date'] + "</mark> - ")
        if paper['comments']:
            doc.comment += " _" + paper['comments'] + "_"
        
        full_md = doc.generate_markdown_text()
        
        full_md += get_markdown_qrcode(paper_id)
        
        # replace citations
        try:
            bibdata = latex_bib.LatexBib.from_doc(doc)
            full_md = latex_bib.replace_citations(full_md, bibdata)
        except Exception as e:
            print(e)
        
        documents.append((paper_id, full_md))
    except Exception as e:
        warnings.warn(latex.LatexWarning(f"{paper_id:s} did not run properly\n" +
                                         str(e)
                                        ))
        failed.append((paper, "latex error " + str(e)))

  0%|          | 0/2 [00:00<?, ?it/s]

Retrieving document from  https://arxiv.org/e-print/2407.21583


extracting tarball to tmp_2407.21583...

 done.


D. Spaeth  ->  D. Spaeth  |  ['D. Spaeth']


Found 112 bibliographic references in tmp_2407.21583/paper.bbl.
Retrieving document from  https://arxiv.org/e-print/2407.21716


extracting tarball to tmp_2407.21716... done.
  0: tmp_2407.21716/natbib.tex, 96 lines
  1: tmp_2407.21716/main_vfinal.tex, 701 lines
  2: tmp_2407.21716/natnotes.tex, 332 lines
  3: tmp_2407.21716/aassymbols.tex, 579 lines


/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Multiple tex files.

  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Found 4 candidates with documentclass definition.
  exec(code_obj, self.user_global_ns, self.user_ns)
/opt/hostedtoolcache/Python/3.9.19/x64/lib/python3.9/site-packages/IPython/core/interactiveshell.py:3550: LatexWarning: Assuming tmp_2407.21716/main_vfinal.tex as main document.
  exec(code_obj, self.user_global_ns, self.user_ns)


### Export the logs

Throughout, we also keep track of the logs per paper. see `logs-{today date}.md` 

In [6]:
import datetime
today = str(datetime.date.today())
logfile = f"_build/html/logs/log-{today}.md"


with open(logfile, 'w') as logs:
    # Success
    logs.write(f'# Arxiv on Deck 2: Logs - {today}\n\n')
    logs.write("""* Arxiv had {0:,d} new papers\n""".format(len(new_papers)))
    logs.write("""    * {0:,d} with possible author matches\n\n""".format(len(candidates)))
    logs.write("## Sucessful papers\n\n")
    display(Markdown("## Successful papers"))
    success = [k[0] for k in documents]
    for candid in candidates:
        if candid['identifier'].split(':')[-1] in success:
            display(candid)
            logs.write(candid.generate_markdown_text() + '\n\n')

    ## failed
    logs.write("## Failed papers\n\n")
    display(Markdown("## Failed papers"))
    failed = sorted(failed, key=lambda x: x[1])
    current_reason = ""
    for paper, reason in failed:
        if 'affiliation' in reason:
            color = 'green'
        else:
            color = 'red'
        data = Markdown(
                paper.generate_markdown_text() + 
                f'\n|<p style="color:{color:s}"> **ERROR** </p>| <p style="color:{color:s}">{reason:s}</p> |'
               )
        if reason != current_reason:
            logs.write(f'### {reason:s} \n\n')
            current_reason = reason
        logs.write(data.data + '\n\n')
        
        # only display here the important errors (all in logs)
        # if color in ('red',):
        display(data)

## Successful papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.21583-b31b1b.svg)](https://arxiv.org/abs/2407.21583) | **Non-radial oscillations mimicking a brown dwarf orbiting the cluster giant NGC 4349 No. 127**  |
|| <mark>D. Spaeth</mark>, et al. |
|*Appeared on*| *2024-08-01*|
|*Comments*| *23 pages, 14 figures. Accepted in A&A. Abstract abridged for arXiv submission*|
|**Abstract**|            Several evolved stars have been found to exhibit long-period radial velocity variations that cannot be explained by planetary or brown dwarf companions. Non-radial oscillations caused by oscillatory convective modes have been put forth as an alternative explanation, but no modeling attempt has yet been undertaken. We provide a model of a non-radial oscillation, aiming to explain the observed variations of the cluster giant NGC 4349 No. 127. The star was previously reported to host a brown dwarf companion, but whose existence was later refuted in the literature. We reanalyzed 58 archival HARPS spectra, acquiring additional activity indicators using the SERVAL and RACCOON pipelines. We searched for periodicity in the indicators and correlations between the indicators and radial velocities. We further present a simulation code able to produce synthetic HARPS spectra, incorporating the effect of non-radial oscillations, and compare the simulated results to the observed variations. We find a positive correlation between chromatic index and radial velocity, along with closed-loop Lissajous-like correlations between radial velocity and each of the spectral line shape indicators (full width at half maximum, and contrast of the cross-correlation function and differential line width). Simulations of a low-amplitude, retrograde, dipole (l = 1, m = 1), non-radial oscillation can reproduce the observed behavior and explain the observables. Photometric variations below the detection threshold of the available ASAS-3 photometry are predicted. The oscillation and stellar parameters are largely in agreement with the prediction of oscillatory convective modes. The periodic variations of the radial velocities and activity indicators, along with the respective phase shifts, measured for the intermediate-mass cluster giant NGC 4349 No. 127, can be explained by a non-radial oscillation.         |

## Failed papers


|||
|---:|:---|
| [![arXiv](https://img.shields.io/badge/arXiv-2407.21716-b31b1b.svg)](https://arxiv.org/abs/2407.21716) | **Revisiting the fundamental metallicity relation with observation and simulation**  |
|| C. Ma, et al. -- incl., <mark>H. Jiang</mark> |
|*Appeared on*| *2024-08-01*|
|*Comments*| *11 pages, 6 figures, ApJL accepted*|
|**Abstract**|            The gas-phase metallicity of galaxies is regulated by multiple astrophysical processes, which makes it a crucial diagnostic of galaxy formation and evolution. Beyond the fundamental mass-metallicity relation, a debate about the secondary galaxy property to predict the metallicity of galaxies arises. Motivated by this, we systematically examine the relationship between gas-phase metallicity and other galaxy properties, i.e. star formation rate (SFR) and galaxy size, in addition to stellar mass in both observation and simulation. We utilize the data from the MaNGA (Mapping Nearby Galaxies at Apache Point Observatory) survey and the TNG50 simulations. We find that the combination of $M_*/R_{\rm e}^\beta$ with $\beta\sim 0.6-1$ is in much stronger correlation to the metallicity than stellar mass alone, regardless of whether the SFR is included or not, in both observation and simulation. This indicates that galaxy size plays a more important role in determining gas-phase metallicity of galaxies than SFR. In addition, the TNG simulation predicts that the SFR, although being a subdominant role, becomes increasingly important in high-$z$ universe. Finally, we speculate that SFR modulates metallicity on the temporal dimension, synchronized with time-varying gas inflows, and galaxy size regulates metallicity on the spatial dimension by affecting the gravitational potential and the mass loading factor.         |
|<p style="color:green"> **ERROR** </p>| <p style="color:green">affiliation error: mpia.affiliation_verifications: 'Heidelberg' keyword not found.</p> |

## Export documents

We now write the .md files and export relevant images

In [7]:
def export_markdown_summary(md: str, md_fname:str, directory: str):
    """Export MD document and associated relevant images"""
    import os
    import shutil
    import re

    if (os.path.exists(directory) and not os.path.isdir(directory)):
        raise RuntimeError(f"a non-directory file exists with name {directory:s}")

    if (not os.path.exists(directory)):
        print(f"creating directory {directory:s}")
        os.mkdir(directory)

    fig_fnames = (re.compile(r'\[Fig.*\]\((.*)\)').findall(md) + 
                  re.compile(r'\<img src="([^>\s]*)"[^>]*/>').findall(md))
    print("found figures", fig_fnames)
    for fname in fig_fnames:
        if 'http' in fname:
            # No need to copy online figures
            continue
        if not os.path.exists(fname):
            print("file not found", fname)
            continue
        print("copying ", fname, "to", directory)
        destdir = os.path.join(directory, os.path.dirname(fname))
        destfname = os.path.join(destdir, os.path.basename(fname))
        try:
            os.makedirs(destdir)
        except FileExistsError:
            pass
        shutil.copy(fname, destfname)
    with open(os.path.join(directory, md_fname), 'w') as fout:
        fout.write(md)
    print("exported in ", os.path.join(directory, md_fname))
    [print("    + " + os.path.join(directory,fk)) for fk in fig_fnames]

In [8]:
for paper_id, md in documents:
    export_markdown_summary(md, f"{paper_id:s}.md", '_build/html/')

found figures ['tmp_2407.21583/./activity_correlation_paper_2x2.png', 'tmp_2407.21583/./super_correlation_12cm.png', 'tmp_2407.21583/./l1_time.png']
copying  tmp_2407.21583/./activity_correlation_paper_2x2.png to _build/html/
copying  tmp_2407.21583/./super_correlation_12cm.png to _build/html/
copying  tmp_2407.21583/./l1_time.png to _build/html/
exported in  _build/html/2407.21583.md
    + _build/html/tmp_2407.21583/./activity_correlation_paper_2x2.png
    + _build/html/tmp_2407.21583/./super_correlation_12cm.png
    + _build/html/tmp_2407.21583/./l1_time.png


## Display the papers

Not necessary but allows for a quick check.

In [9]:
[display(Markdown(k[1])) for k in documents];

<div class="macros" style="visibility:hidden;">
$\newcommand{\ensuremath}{}$
$\newcommand{\xspace}{}$
$\newcommand{\object}[1]{\texttt{#1}}$
$\newcommand{\farcs}{{.}''}$
$\newcommand{\farcm}{{.}'}$
$\newcommand{\arcsec}{''}$
$\newcommand{\arcmin}{'}$
$\newcommand{\ion}[2]{#1#2}$
$\newcommand{\textsc}[1]{\textrm{#1}}$
$\newcommand{\hl}[1]{\textrm{#1}}$
$\newcommand{\footnote}[1]{}$</div>



<div id="title">

# Non-radial oscillations mimicking a brown dwarf orbiting the cluster giant NGC 4349 No. 127

</div>
<div id="comments">

[![arXiv](https://img.shields.io/badge/arXiv-2407.21583-b31b1b.svg)](https://arxiv.org/abs/2407.21583)<mark>Appeared on: 2024-08-01</mark> -  _23 pages, 14 figures. Accepted in A&A. Abstract abridged for arXiv submission_

</div>
<div id="authors">

<mark>D. Spaeth</mark>, et al.

</div>
<div id="abstract">

**Abstract:** Several evolved stars have been found to exhibit long-period radial velocity variations that cannot be explained by planetary or brown dwarf companions. Non-radial oscillations caused by oscillatory convective modes have been put forth as an alternative explanation, but no modeling attempt has yet been undertaken. We provide a model of a non-radial oscillation, aiming to explain the observed variations of the cluster giant NGC 4349 No. 127. The star was previously reported to host a brown dwarf companion, but whose existence was later refuted in the literature. We reanalyzed 58 archival HARPS spectra of the intermediate-mass giant NGC 4349 No. 127. We reduced the spectra using the SERVAL and RACCOON pipelines, acquiring additional activity indicators. We searched for periodicity in the indicators and correlations between the indicators and radial velocities. We further present a simulation code able to produce synthetic HARPS spectra, incorporating the effect of non-radial oscillations, and compare the simulated results to the observed variations. We discuss the possibility that non-radial oscillations cause the observed variations. We find a positive correlation between chromatic index and radial velocity, along with closed-loop Lissajous-like correlations between radial velocity and each of the spectral line shape indicators (full width at half maximum, and contrast of the cross-correlation function and differential line width). Simulations of a low-amplitude, retrograde, dipole ( $l=1, m=1$ ), non-radial oscillation can reproduce the observed behavior and explain the observables. Photometric variations below the detection threshold of the available ASAS-3 photometry are predicted. The oscillation and stellar parameters are largely in agreement with the prediction of oscillatory convective modes. The periodic variations of the radial velocities and activity indicators, along with the respective phase shifts, measured for the intermediate-mass cluster giant NGC 4349 No. 127, can be explained by a non-radial oscillation.

</div>

<div id="div_fig1">

<img src="tmp_2407.21583/./activity_correlation_paper_2x2.png" alt="Fig8" width="100%"/>

**Figure 8. -** Observed (data points) and simulated (lines) correlations between the activity indicators and the RVs for 44 spectra taken prior to the HARPS fiber change. For each panel, the mean (RV and any of the indicators, respectively) was subtracted.
    Each data point is color-coded with the phase according to the best RV period $P=674.0 \mathrm{d}$.
    While the CRX (top left) shows a significant positive correlation with the RVs (r=0.54, $p(_F_\textrm{-test})=0.002\%$), dLW, FWHM, and contrast of the CCF are correlated with the RVs in a closed-loop behavior.
    We plot the linear (CRX) and elliptical fits (dLW, FWHM, contrast) to the simulated data points for the best model of a $l=1, m=1$ oscillation mode as solid lines applying the same color-coding. As a linear relationship is predicted between CRX and RV, we plot the ascending and descending phase relations on top of the black fit to the simulations. The simulated ellipses can closely reproduce the observed behavior including the amplitudes, phases, and directions of correlation. (*fig:corr*)

</div>
<div id="div_fig2">

<img src="tmp_2407.21583/./super_correlation_12cm.png" alt="Fig11" width="100%"/>

**Figure 11. -** Correlation plots between the activity indicators and the RVs for modes with $l=1$(left column), $m=l$(center column), and $m=-l$(right column). Linear correlations were fitted for the correlation between CRX and RV. For the line shape diagnostics, ellipses were fitted when appropriate. Arrows indicate the temporal dependence of the ellipses. The individual modes are specified in the legend at the top of each column. The simulated dLW variations were rescaled with a common factor of $0.1$. (*fig:supercorrelation*)

</div>
<div id="div_fig3">

<img src="tmp_2407.21583/./l1_time.png" alt="Fig10" width="100%"/>

**Figure 10. -** RVs and activity indicators of dipole oscillation modes plotted against time at inclination angle $i=45◦$. Sinusoids were fitted to all time series except for the FWHM, dLW, and contrast in case of $m=0$, for which the simulated points were interconnected. While all three modes cause similar RV variations (top left panel), the phase relations between the RVs and the line shape indicators, as well as their amplitudes, are notably different. The CRX (bottom left) shows a similar behavior for all three modes. The dLW variations were rescaled with a common factor of $0.1$. (*fig:l1_time*)

</div><div id="qrcode"><img src=https://api.qrserver.com/v1/create-qr-code/?size=100x100&data="https://arxiv.org/abs/2407.21583"></div>

# Create HTML index

In [10]:
from datetime import datetime, timedelta, timezone
from glob import glob
import os

files = glob('_build/html/*.md')
days = 7
now = datetime.today()
res = []
for fk in files:
    stat_result = os.stat(fk).st_ctime
    modified = datetime.fromtimestamp(stat_result, tz=timezone.utc).replace(tzinfo=None)
    delta = now.today() - modified
    if delta <= timedelta(days=days):
        res.append((delta.seconds, fk))
res = [k[1] for k in reversed(sorted(res, key=lambda x:x[1]))]
npub = len(res)
print(len(res), f" publications files modified in the last {days:d} days.")
# [ print('\t', k) for k in res ];

150  publications files modified in the last 7 days.


In [11]:
import datetime
from glob import glob

def get_last_n_days(lst, days=1):
    """ Get the documents from the last n days """
    sorted_lst = sorted(lst, key=lambda x: x[1], reverse=True)
    for fname, date in sorted_lst:
        if date >= str(datetime.date.today() - datetime.timedelta(days=days)):
            yield fname

def extract_appearance_dates(lst_file):
    dates = []

    def get_date(line):
        return line\
            .split('Appeared on:')[-1]\
            .split('</mark>')[0].strip()

    for fname in lst:
        with open(fname, 'r') as f:
            found_date = False
            for line in f:
                if not found_date:
                    if "Appeared on" in line:
                        found_date = True
                        dates.append((fname, get_date(line)))
                else:
                    break
    return dates

from glob import glob
lst = glob('_build/html/*md')
days = 7
dates = extract_appearance_dates(lst)
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last {days:d} days.")

4  publications in the last 7 days.


In [12]:
def create_carousel(npub=4):
    """ Generate the HTML code for a carousel with `npub` slides """
    carousel = ["""  <div class="carousel" """,
                """       data-flickity='{ "autoPlay": 10000, "adaptiveHeight": true, "resize": true, "wrapAround": true, "pauseAutoPlayOnHover": true, "groupCells": 1 }' id="asyncTypeset">"""
                ]
    
    item_str = """    <div class="carousel-cell"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        carousel.append(item_str.format(k=k))
    carousel.append("  </div>")
    return '\n'.join(carousel)

def create_grid(npub=4):
    """ Generate the HTML code for a flat grid with `npub` slides """
    grid = ["""  <div class="grid"> """,
                ]
    
    item_str = """    <div class="grid-item"> <div id="slide{k}" class="md_view">Content {k}</div> </div>"""
    for k in range(1, npub + 1):
        grid.append(item_str.format(k=k))
    grid.append("  </div>")
    return '\n'.join(grid)

In [13]:
carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "7-day archives" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
with open("_build/html/index_7days.html", 'w') as fout:
    fout.write(page)

In [14]:
# redo for today
days = 1
res = list(get_last_n_days(dates, days))
npub = len(res)
print(len(res), f" publications in the last day.")

carousel = create_carousel(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("daily_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- carousel:s --%}", carousel)\
               .replace("{%-- suptitle:s --%}",  "Daily" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(carousel, docs, slides)
# print(page)
with open("_build/html/index_daily.html", 'w') as fout:
    fout.write(page)

1  publications in the last day.


In [15]:
# Create the flat grid of the last N papers (fixed number regardless of dates)
from itertools import islice 

npub = 6
res = [k[0] for k in (islice(reversed(sorted(dates, key=lambda x: x[1])), 6))]
print(len(res), f" {npub} publications selected.")

grid = create_grid(npub)
docs = ', '.join(['"{0:s}"'.format(k.split('/')[-1]) for k in res])
slides = ', '.join([f'"slide{k}"' for k in range(1, npub + 1)])

with open("grid_template.html", "r") as tpl:
    page = tpl.read()
    page = page.replace("{%-- grid-content:s --%}", grid)\
               .replace("{%-- suptitle:s --%}",  f"Last {npub:,d} papers" )\
               .replace("{%-- docs:s --%}", docs)\
               .replace("{%-- slides:s --%}", slides)
    
# print(grid, docs, slides)
# print(page)
with open("_build/html/index_npub_grid.html", 'w') as fout:
    fout.write(page)

6  6 publications selected.
